In [54]:
import os
import json
from collections import deque
from typing import Dict, List, Optional, Any, Union
from pydantic import BaseModel

from langchain.agents import ZeroShotAgent, Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain import OpenAI, SerpAPIWrapper, LLMChain, PromptTemplate
from langchain.chat_models import ChatOpenAI


from langchain.vectorstores import Chroma
from langchain.docstore import InMemoryDocstore

from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
import chromadb
import re


# Get human objective
# refine it via human in the loop and via adding context (short/long term memory search) -> the output is a json dict of the design
# build the modules and test benches usiiing short and long term memory

In [2]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["SERPAPI_API_KEY"] = os.getenv("SERPAPI_API_KEY")

In [3]:
llm = ChatOpenAI(model='gpt-4', temperature=0)
bigllm = ChatOpenAI(model='gpt-4-1106-preview', temperature=0)

In [42]:
from FPGA_AGI.tools import *
from FPGA_AGI.agents import *
from FPGA_AGI.utils import *

In [22]:
OBJECTIVE = 'This design will be implemented on a zync ultrascale device. design a state machine with 4 states: Add, Subtract, Multiply, Divide. the state transitions are controlled via an axi-lite connection that comes from the PS side. The Multiply\Divide state can only go to the add or subtract state (depending on the control) and the Add\Subtract states can only go the the multiply or divide states. Implement this in system verilog.'
##########prompt1
tools = [
    web_search_tool,
    document_search_tool,
    human_input_tool
]
requirement_writing_agent = RequirementAgentExecutor.from_llm_and_tools(llm=bigllm, tools=tools, verbose=True)

requirement = requirement_writing_agent.run(OBJECTIVE) 



> Entering new AgentExecutor chain...
Thought: To provide a comprehensive explanation of the key functions, goals, and specific requirements for the FPGA design project, I need to understand the Zynq UltraScale architecture, the AXI-Lite interface, and the state machine design in SystemVerilog. I will start by gathering information on the Zynq UltraScale device and the AXI-Lite protocol.

Action: Doc_Search
Action Input: Zynq UltraScale architecture overview
Observation: {'question': 'Zynq UltraScale architecture overview', 'answer': 'The Zynq UltraScale+ MPSoC architecture includes a quad-core ARM Cortex-A53 processor, a dual-core ARM Cortex-R5 real-time processor, a Mali-400 MP2 graphics processing unit, and programmable logic. It also supports high-level operating systems like Linux and offers various peripherals and interfaces such as USB, Ethernet, HDMI, and PCIe. Additionally, some UltraScale architecture-based devices include integrated blocks for the Interlaken chip-to-chip i

In [24]:

project_details = extract_project_details(requirement)
project_details



ProjectDetails(goals='- Design a state machine with four states: Add, Subtract, Multiply, Divide. - Ensure state transitions are controlled via an AXI-Lite connection from the PS side. - Implement the design in SystemVerilog on a Zynq UltraScale device.', requirements='- The state machine must be able to transition between states based on control signals received through the AXI-Lite interface. - The Multiply and Divide states can only transition to the Add or Subtract states. - The Add and Subtract states can only transition to the Multiply or Divide states. - Use SystemVerilog constructs such as `case` and `if` statements to manage state transitions and outputs. - Separate combinational and sequential logic into different `always` blocks. - Use `parameter` or `` `define `` to declare state constants for the state machine. - Ensure that the design is synthesizable and can be implemented on the Zynq UltraScale FPGA.', constraints='- The design must conform to the Zynq UltraScale archit

In [25]:
###Prompt 2 with agent
tools = [
    web_search_tool,
    document_search_tool,
]

agent_executor = ModuleAgentExecutor.from_llm_and_tools(llm=bigllm, tools=tools, verbose=True)
module_list = agent_executor.run(Goals=project_details.goals, Requirements=project_details.requirements, Constraints=project_details.constraints)



> Entering new AgentExecutor chain...
Thought 1: To satisfy the goals and requirements, I need to identify the necessary modules for the FPGA-based hardware design project, including a top module that encapsulates all other modules.
Thought 2: I should further refine thought 1 by considering the specific functionalities required by each module, such as state machine logic, AXI-Lite interface, and any additional support modules.
Thought 3: I should think about what to do to gather information on the typical modules required for such a design, especially focusing on the Zynq UltraScale architecture.
Action: Doc_Search
Action Input: Zynq UltraScale FPGA design modules

Observation:{'question': 'Zynq UltraScale FPGA design modules', 'answer': 'The Zynq UltraScale FPGA design modules include Programmable Logic (PL), Configurable Logic Blocks (CLB), Look-up tables (LUT), Flip-flops, Cascadable adders, 36Kb Block RAM, True dual-port, UltraRAM, Error checking and correction, DSP Blocks, Prog

In [26]:
module_list

'```json\n[\n  {\n    "Module_Name": "Top_Module",\n    "ports": ["AXI_Lite_signals", "State_Machine_signals", "Clock", "Reset"],\n    "description": "The top-level module that encapsulates the state machine, AXI-Lite interface, and any additional support modules. It manages the overall communication and synchronization between the PS and the PL.",\n    "connections": ["State_Machine_Module", "AXI_Lite_Interface_Module", "Support_Modules"],\n    "hdl_language": "SystemVerilog"\n  },\n  {\n    "Module_Name": "State_Machine_Module",\n    "ports": ["Control_Signals", "State_Outputs", "Clock", "Reset"],\n    "description": "This module implements the state machine logic for the Add, Subtract, Multiply, and Divide operations. It transitions between states based on control signals received from the AXI-Lite interface and performs the associated operations.",\n    "connections": ["AXI_Lite_Interface_Module"],\n    "hdl_language": "SystemVerilog"\n  },\n  {\n    "Module_Name": "AXI_Lite_Interf

In [27]:
module_list = extract_json_from_string(module_list)

In [39]:
############## Agent based HDL generation
tools = [
    #web_search_tool,
    document_search_tool,
]
hdl_agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)


In [40]:
codes = []
module_list_str = [str(item) for item in module_list]
#current_modules_verified = json.loads(current_modules_verified)
for module in module_list:
   code = hdl_agent_executor.run(Goals=project_details.goals, Requirements=project_details.requirements, Constraints=project_details.constraints, module_list='\n'.join(module_list_str), codes='\n'.join(codes), module=str(module))
   codes.append(code)




> Entering new AgentExecutor chain...
Thought: To implement the 'Top_Module', I need to understand the AXI-Lite protocol and how to interface it with a state machine in SystemVerilog. I should also consider the structure of the top-level module, which will instantiate and connect the submodules.

Action: Doc_Search
Action Input: AXI-Lite interface SystemVerilog example


Observation:{'question': 'AXI-Lite interface SystemVerilog example\n', 'answer': 'There is an example of an AXI4-Lite interface in the document "HLS-tutorial.pdf" and it is used in the core design described in the document "matrixmult-zynq-hls.pdf". The matching HLS bus interface channel for the AXI4M_bus_port is shown in the document "ug902-vivado-high-level-synthesis.pdf".\n', 'sources': 'HLS-tutorial.pdf, matrixmult-zynq-hls.pdf, ug902-vivado-high-level-synthesis.pdf', 'source_documents': [Document(page_content='optimal balance of area and performance to  satisfy your particular requirements.\nStep 3: Implementing

In [41]:
save_solution(codes, 0)

In [43]:
module["hdl_language"]

'SystemVerilog'

## FPGA-AGI

In [55]:
from pydantic import BaseModel
import warnings

class FPGA_AGI(BaseModel):
    verbose: bool = True
    requirement_agent_executor: RequirementAgentExecutor = None
    module_agent_exxecutor: ModuleAgentExecutor = None
    hdl_agent_executor: HdlAgentExecutor = None

    def __init__(self, **data):
        super().__init__(**data)
        if self.requirement_agent_executor is None:
            warnings.warn("RequirementAgentExecutor is not set. Please use the from_llm class method to properly initialize it.", UserWarning)

    @classmethod
    def from_llm(cls, llm, verbose=True):
        requirement_agent_executor = RequirementAgentExecutor(llm=llm)
        return cls(verbose=verbose, requirement_agent_executor=requirement_agent_executor)

# Example of using the class method
llm_instance = ...  # however you initialize or obtain your llm
fpga_agi_instance = FPGA_AGI.from_llm(llm=llm_instance, verbose=False)

# Direct instantiation will issue a warning
fpga_agi_direct = FPGA_AGI(verbose=False)


test = FPGA_AGI()

In [58]:
test.verbose

True

OBJECTIVE = "Elaborate the best definition for sustainable finance"
llm = ChatOpenAI(model='gpt-4', temperature=0)
# Logging of LLMChains
verbose=False
# If None, will keep on going forever
max_iterations: Optional[int] = 7
baby_agi = BabyAGI.from_llm(
    llm=llm,
    vectorstore=vectorstore,
    verbose=verbose,
    max_iterations=max_iterations
)

baby_agi({"objective": OBJECTIVE})